<a href="https://colab.research.google.com/github/Shibu4064/ML_Project/blob/main/sagor_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
!pip install transformers datasets
!pip install transformers
!pip install transformers[torch]
!pip install transformers[tf-cpu]
!pip install transformers[flax]
!pip install torch
!pip install tensorflow
!pip install evaluate

In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/EMNLP/train.csv') # train set
val_df = pd.read_csv('/content/drive/MyDrive/EMNLP/dev.csv') # validation set
#train_df=pd.read_csv('/content/drive/MyDrive/EMNLP/df12_cleaned.csv')

In [ ]:
print(len(train_df))

2700


In [ ]:
train_df.columns

Index(['text', 'label'], dtype='object')

In [ ]:
train_df.head()

,text,label
0,যে দেশে সন্ত্রাসরা দেশ চালায়সে দেশে শান্তি কিভ...,1
1,এই বিচার শেষ বিচার নয়।আসল বিচার হবে আল্লাহর আদ...,0
2,আরব দেশগুলোকে বলব ভারতের সাথে সব ব্যবসা বাণিজ্...,2
3,দেশটা সুস্থ নাই,0
4,আপনার কথা দুঃখ জনক আগে বিডিও থাকলে কেন ধরা হলন...,0


In [ ]:
from sklearn.model_selection import train_test_split
def pandaToList(dataset):
    text_list = []
    label_list = []
    for i in range(len(dataset['text'])):
      text_list.append(dataset['text'][i])
      label_list.append(int(dataset['label'][i]))
    return text_list, label_list

train_texts, train_labels =  pandaToList(train_df)
val_texts, val_labels = pandaToList(val_df)

In [ ]:
!pip uninstall jaxlib

Found existing installation: jaxlib 0.4.13
Uninstalling jaxlib-0.4.13:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/jaxlib-0.4.13.dist-info/*
    /usr/local/lib/python3.10/dist-packages/jaxlib/*
Proceed (Y/n)? y
  Successfully uninstalled jaxlib-0.4.13


In [ ]:
pip install -U jax jaxlib

  Using cached jax-0.4.14-py3-none-any.whl
  Using cached jaxlib-0.4.14-cp310-cp310-manylinux2014_x86_64.whl (73.7 MB)
  Attempting uninstall: jax
    Found existing installation: jax 0.4.13
    Uninstalling jax-0.4.13:
      Successfully uninstalled jax-0.4.13


In [ ]:
tokenizer_name = 'sagorsarker/bangla-bert-base'
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
import torch

class ViolenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ViolenceDataset(train_encodings, train_labels)
val_dataset = ViolenceDataset(val_encodings, val_labels)


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy, AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertTokenizer, BertModel
training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   #output_dir= output_dir,
   eval_steps = 250, # Evaluation and Save happens every 250 steps
   save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.

   learning_rate=1e-5,
   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   num_train_epochs=50,
   weight_decay=0.01,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True)

model_name = 'sagorsarker/bangla-bert-base'
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3) ## change here for using another pretrained model

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
250,No log,1.111387,0.588722,0.420800,0.396794,0.356513
500,1.379300,1.276353,0.612782,0.415494,0.432407,0.402918
750,1.379300,1.290256,0.636842,0.594341,0.473151,0.464326
1000,1.243400,1.561619,0.646617,0.633404,0.558397,0.567530
1250,1.243400,1.558452,0.653383,0.598530,0.546364,0.555288
1500,1.376200,1.507686,0.683459,0.662014,0.565872,0.587805
1750,1.376200,1.539241,0.687970,0.693530,0.568038,0.592169
2000,1.617000,1.392472,0.672932,0.663756,0.564574,0.575689
2250,1.617000,1.531875,0.687218,0.680188,0.568299,0.592977
2500,1.317000,1.450568,0.684962,0.663065,0.576881,0.593979


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=4250, training_loss=1.2588113008386947, metrics={'train_runtime': 982.3091, 'train_samples_per_second': 137.431, 'train_steps_per_second': 137.431, 'total_flos': 408416774881500.0, 'train_loss': 1.2588113008386947, 'epoch': 1.57})

In [ ]:
trainer.predict(val_dataset)

PredictionOutput(predictions=array([[ 4.694577 , -2.106599 , -2.7200754],
       [ 4.5345383, -2.0135581, -2.7230957],
       [ 4.8577785, -2.3583724, -2.5457942],
       ...,
       [ 4.677658 , -2.1518087, -2.6738553],
       [ 1.6586131,  1.7194527, -3.174642 ],
       [-3.22201  ,  4.844215 , -1.9898009]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 1, 0]), metrics={'test_loss': 1.7819310426712036, 'test_accuracy': 0.6977443609022557, 'test_precision': 0.6653532338308458, 'test_recall': 0.6517381261927526, 'test_f1': 0.6545709703211143, 'test_runtime': 22.2225, 'test_samples_per_second': 59.849, 'test_steps_per_second': 59.849})

In [ ]:
output_dir = "/content/drive/MyDrive/VITD-Models/sagorbert"

In [ ]:
trainer.save_model(f"{output_dir}") # saving the model to drive.